In [1]:
import numpy as np
import keras
import os,cv2,sys
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,AveragePooling2D,Dropout
#from skimage.feature import local_binary_pattern
#from scipy.stats import itemfreq
from sklearn.preprocessing import normalize

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_labels = 7
width, height = 48, 48

In [3]:
def facedetection(filename):
    face_cascade = cv2.CascadeClassifier('C:\\Users\\Shrey Malvi\\Downloads\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
    cv2.imwrite('tempimg.jpg',filename)
    
    img1=cv2.imread('tempimg.jpg')
    faces = face_cascade.detectMultiScale(img1, 1.3, 5)
    index=0
    for (x,y,w,h) in faces:
        cropped = img1[y :y +  h , x : x + w]
        return cropped

In [4]:
def featureextraction(scaled):
    y,x=scaled.shape
    gx=np.zeros((x,y))
    scaled = np.lib.pad(scaled, 1, 'constant', constant_values=0)
    gy = np.zeros((x, y))
    for i in range(y):
        a = np.convolve(scaled[i - 1, :], [1, 0, -1], 'valid')
        b = np.convolve(scaled[i, :], [2, 0, -2], 'valid')
        c = np.convolve(scaled[i + 1, :], [1, 0, -1], 'valid')
        gx[i, :] = np.sum([a, b, c], axis=0)
    for j in range(x):
        a = np.convolve(scaled[:, j - 1], [1, 0, -1], 'valid')
        b = np.convolve(scaled[:, j], [2, 0, -2], 'valid')
        c = np.convolve(scaled[:, j + 1], [1, 0, -1], 'valid')
        gy[:, j] = np.sum([a, b, c], axis=0)
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    angle=angle%180;
    for i in range(0,x):
        for j in range(0,y):
            if(angle[i,j]<=20):
                angle[i,j]=0
            elif(angle[i,j]<=40):
                angle[i,j]=1
            elif(angle[i,j]<=60):
                angle[i,j]=2
            elif(angle[i,j]<=80):
                angle[i,j]=3
            elif(angle[i,j]<=100):
                angle[i,j]=4
            elif(angle[i,j]<=120):
                angle[i,j]=5
            elif(angle[i,j]<=140):
                angle[i,j]=6
            elif(angle[i,j]<=160):
                angle[i,j]=7
            elif(angle[i,j]<=180):
                angle[i,j]=8
    angle=np.int64(angle)
    mag=np.float64(mag)
    bin_n=9
    cells=8
    blocks=2
    width = scaled.shape[0]
    height = scaled.shape[1]
    angle=np.int64(angle)
    mag=np.float64(mag)
    epsilon = sys.float_info.epsilon
    x=56
    y=56
    yd= np.int64(((height - (cells * blocks)) / cells) + 1)
    xd= np.int64(((width - (cells * blocks)) / cells) + 1)
    c=0
    histograms = []
    for i in range(0,xd,1):
        for j in range(0,yd,1):
            b_mag=mag[i*cells:i*cells+cells*blocks,j*cells:j*cells+cells*blocks]
            b_angle=angle[i*cells:i*cells+cells*blocks,j*cells:j*cells+cells*blocks]
            tempHists = []
            sumHists = np.zeros((9,))
            for m in range(0,blocks*cells,cells):
                for n in range(0,blocks*cells,cells):
                    c_mag=b_mag[m:m+cells,n:n+cells]
                    c_angle=b_angle[m:m+cells,n:n+cells]
                    tempHists.append(np.bincount(c_angle.ravel(), c_mag.ravel(), bin_n))
                    sumHists = np.sum([sumHists, np.bincount(c_angle.ravel(),c_mag.ravel(), bin_n)], axis=0)
            sumHists = np.sum(sumHists)
            for hist in tempHists:
                histograms.append(np.divide(hist, np.sqrt(np.add(np.square(sumHists), np.square(epsilon)))))        
    hist = np.hstack(histograms)
    return hist


In [5]:
def lbp(scaled):
    #im_gray = cv2.cvtColor(scaled, cv2.COLOR_BGR2GRAY)
    radius = 5
    no_points = 8 * radius
    lbp = local_binary_pattern(scaled, no_points, radius, method='uniform')
    x = itemfreq(lbp.ravel())
    hist = x[:, 1]/sum(x[:, 1])
    print(len(hist))
    return hist

In [6]:
data = pd.read_csv('./fer2013.csv')

In [20]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [25]:
pixels = data['pixels'].tolist() 
faces = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] 
    face=np.asarray(face).reshape(width,height)
    input_fx=featureextraction(face)
    faces.append(input_fx)
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1) 
emotions = pd.get_dummies(data['emotion']).as_matrix()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.


In [22]:
print(faces.shape)
#faces=faces.reshape((35887,900))

(35887, 1)


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [25]:
print(X_train.shape)

(29068, 1)


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation
#mlp = MLPClassifier(hidden_layer_sizes=(128,64,32),activation='relu',learning_rate_init=0.001,max_iter=200,batch_size=75,verbose=2)
#mlp.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
#mlp.fit(X_train,y_train)
model = Sequential()
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=100, batch_size=50)

Epoch 1/100
29068/29068 [==============================] - 3s 117us/step - loss: 1.6337 - acc: 0.3554
Epoch 2/100
29068/29068 [==============================] - 2s 60us/step - loss: 1.5213 - acc: 0.4110
Epoch 3/100
29068/29068 [==============================] - 2s 59us/step - loss: 1.4768 - acc: 0.4270
Epoch 4/100
29068/29068 [==============================] - 2s 59us/step - loss: 1.4445 - acc: 0.4386
Epoch 5/100
29068/29068 [==============================] - 2s 63us/step - loss: 1.4139 - acc: 0.4524
Epoch 6/100
29068/29068 [==============================] - 2s 60us/step - loss: 1.3854 - acc: 0.4629
Epoch 7/100
29068/29068 [==============================] - 2s 72us/step - loss: 1.3520 - acc: 0.4782
Epoch 8/100
29068/29068 [==============================] - 2s 64us/step - loss: 1.3232 - acc: 0.4936
Epoch 9/100
29068/29068 [==============================] - 2s 80us/step - loss: 1.2943 - acc: 0.5051
Epoch 10/100
29068/29068 [==============================] - 2s 58us/step - loss: 1.2656 - 

In [29]:
predictions=model.predict(X_test)
acc = model.evaluate(X_test, y_test)
print('Test Acc:', acc[1])
print(np.round(predictions))

3589/3589 [==============================] - 0s 29us/step
Test Acc: 0.42853162444112824
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [21]:
model.save('my_model.h5') 

In [1]:
#model = load_model('TL2.h5')

In [5]:
from keras.models import load_model
import cv2
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}

model = load_model('Tl2.h5')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier('C:\\Users\\Shrey Malvi\\Downloads\\opencv\\build\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = cv2.resize(roi_gray, (48, 48))
        #cropped_img = cv2.resize(roi_gray, (48, 48))
        #face=np.asarray(cropped_img).reshape(1,48,48,3)
        face = np.repeat(cropped_img[..., np.newaxis], 3, -1)
        face=np.reshape(face,(1,48,48,3))
        #cropped_img1=featureextraction(cropped_img)
        #yp=cropped_img1.reshape((1,900))
        prediction = model.predict(face)
        #cropped_img1=featureextraction(cropped_img)
        #yp=cropped_img1.reshape((1,900))
        #prediction = model.predict(yp)
        cv2.putText(frame, emotion_dict[int(np.argmax(prediction))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [30]:
img=cv2.imread('0.png')

im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
im_gray=cv2.resize(im_gray,(48,48))
print(im_gray.shape)
radius = 3
no_points = 8 * radius
lbp = local_binary_pattern(im_gray, no_points, radius, method='uniform')
x = itemfreq(lbp.ravel())
hist = x[:, 1]/sum(x[:, 1])
print(hist)

(48, 48)
[0.0390625  0.01866319 0.01953125 0.01041667 0.02256944 0.01909722
 0.02690972 0.02907986 0.03472222 0.04730903 0.06684028 0.07118056
 0.06987847 0.05208333 0.02430556 0.02126736 0.01605903 0.0078125
 0.00737847 0.00954861 0.00694444 0.01302083 0.01432292 0.015625
 0.02213542 0.31423611]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  if __name__ == '__main__':


In [10]:
import cv2
import numpy as np
img=cv2.imread('1.png')
height=48
width=48
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
scaled = cv2.resize(gray, (height, width))
sift = 
kp, des = sift.detectAndCompute(scaled,None)



31


In [12]:
print(des.shape)

(31, 128)


In [21]:
def sift(scaled):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(scaled,None)
    return(des.ravel())